## Erkenntis / Aussagen
* Kfz-Fahrten machen ca. 50% aller Wege aus
* Kfz-Fahrten werden hauptsächlich für kurze Wege benutzt, ca. 38% der Fahrten <= 5km


## Ideen
* Fahrten auf Karte einzeichen mit Anfragen über Google Maps Routing
* Histogramm der Fahrten stadtteilaufgelöst
* Pro Stadtteil die mittleren Wege mit bestimmten Verkehrsmitteln
* Chord-Plot wer fährt von wo nach wo? Wie häufig werden Fahrten zwischen Bezirken gemacht?
* Sankey-Diagramm der Fahrten 
    * links Stadtbezirke und rechts Stadtbezirke
    * ähnlich zu Wählerwanderungs-Plots



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def read_haushalte():
    df = pd.read_json('./data/haushalte.json')
    
    
    rename = {
        'Fußweg in Minuten zur nächsten Haltestelle von Bus': 'distance_bus',
        'Fußweg in Minuten zur nächsten Haltestelle von U- / Straßen- / Stadtbahn': 'distance_metro',
        'Fußweg in Minuten zur nächsten Haltestelle von S-Bahn / Nahverkehr': 'distance_train',
    }

    df.rename(columns=rename, inplace=True)
    
    numeric_columns = [
        'distance_bus',
        'distance_metro',
        'distance_train',
    ]
    
    persons = df['Anzahl der im Haushalt ständig lebenden Personen']
    
    df['cars_per_person'] = df['Anzahl der KFZ'] / persons
    df['bicycles_per_person'] = df['Anzahl der betriebsbereiten Fahrräder im HH'] / persons
    df['motorbikes_per_person'] = df['Anzahl der motorisierten Zweiräder im HH'] / persons
    
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    col = 'Wohnort Stadtbezirk Text'
    for before, after in {'ü': 'ue', 'ö': 'oe', '-': ' '}.items():
        df[col] = df[col].str.replace(before, after)
    
    return df


In [ ]:
def read_wege():
    wege = pd.read_csv("./data/wege.csv", sep=";", decimal=",")
    return wege

In [ ]:
def read_personen():
    df = pd.read_json('./data/personen.json')
    
    return df

In [ ]:
wege = read_wege()
haushalte = read_haushalte()
personen = read_personen()

### Zurückgelegte Wege nach Verkehrsmittel

In [ ]:
wege_nach_verkehrsmittel = wege.groupby("Hauptverkehrsmittel Text")["Entfernung in [km] aus Routing GoogleMaps"].describe()
wege_nach_verkehrsmittel

In [ ]:
ax = (
    wege_nach_verkehrsmittel["count"] *
    wege_nach_verkehrsmittel["mean"]
).sort_values().plot.barh(figsize=(8, 8))

ax.set_xlabel("Gesamte Strecke / km")

In [ ]:
ax = (
    wege_nach_verkehrsmittel["50%"]
).sort_values().plot.barh(figsize=(8, 8))

ax.set_xlabel("Mittlere Strecke pro Fahrt / km")

In [ ]:
ax = (
    wege_nach_verkehrsmittel["count"]
).sort_values().plot.barh(figsize=(8, 8))

ax.set_xlabel("Anzahl Fahrten")

### Verteilung der Wegsrecken nach Verkehrsmittel

#### Gruppierung
* (U-Bahn, S-, Straßen-, Stadtbahn & Bus) zu OPNV
* (Fahrrad & E-Bike) zu Fahrrad 
* (Kfz Selbstfahrer, Mitfahrer, Taxi, Motorrad) zu Kfz

In [ ]:
wege["Hauptverkehrsmittel Gruppierung"] = wege["Hauptverkehrsmittel Text"]
wege.loc[wege["Hauptverkehrsmittel Gruppierung"].isin(["U-Bahn / Straßen- /Stadtbahn", "S-Bahn / Nahverkehr", "Bus"]), "Hauptverkehrsmittel Gruppierung"] = "Nahverkehr"
wege.loc[wege["Hauptverkehrsmittel Gruppierung"].isin(["Fahrrad", "E-Bike / Pedelec"]), "Hauptverkehrsmittel Gruppierung"] = "Fahrrad"
wege.loc[wege["Hauptverkehrsmittel Gruppierung"].isin(["Kfz Selbstfahrer", "Kfz Mitfahrer", "Taxi", "Motorrad"]), "Hauptverkehrsmittel Gruppierung"] = "Kfz" # Taxi, 

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=200)
for mittel in wege["Hauptverkehrsmittel Gruppierung"].unique():
    print(mittel)
    wege_mittel = wege[wege["Hauptverkehrsmittel Gruppierung"] == mittel]["Entfernung in [km] aus Routing GoogleMaps"]
    (wege_mittel / wege_mittel.quantile(0.95)).hist(ax=ax, bins=np.linspace(0, 1, 51), density=True, label=mittel, histtype="step")
    
ax.legend()

##### Verteilung der Haushalte nach Stadtbezirken

In [ ]:
# Haushalts-Stadtbezirk für Wege aus Haushalten, join über Personen-ID
# Join (Personen.HaushaltID, Haushalt.ID + Wohnort)
# Join (Wege.PersonenID, Personen.ID + HaushaltID)

personen_mit_stadtbezirk = haushalte[
    ["ID", "Wohnort Stadtbezirk Text"]
].join(personen[["ID", "HaushaltID"]].set_index("HaushaltID"), on="ID", rsuffix="_personen")

wege_mit_stadtbezirk = wege.join(personen_mit_stadtbezirk.set_index("ID_personen")["Wohnort Stadtbezirk Text"], on="Personen ID")

In [ ]:
verkehrsmittel = ["Kfz", "Fahrrad", "Nahverkehr", "Fuss", "Fernverkehr", "Sonstiges"]

In [ ]:
mittel_to_color = {
    mittel : f"C{i}"
    for i, mittel in enumerate(verkehrsmittel)
}

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=200)
for mittel in verkehrsmittel:
    print(mittel)
    wege_mittel = wege[wege["Hauptverkehrsmittel Gruppierung"] == mittel]["Entfernung in [km] aus Routing GoogleMaps"]
    (wege_mittel).hist(ax=ax, bins=np.linspace(0, 20, 41), label=mittel, histtype="step", color=mittel_to_color[mittel])
    
ax.set(ylabel="Anzahl Fahrten", xlabel="Zurückgelegte Kilometer")
ax.legend()

Filtern nach Stadtbezirk

In [ ]:
stadtbezirke = sorted(wege_mit_stadtbezirk["Wohnort Stadtbezirk Text"].dropna().unique())

In [ ]:
fig, axes = plt.subplots(figsize=(14, 10), dpi=200, ncols=4, nrows=3, constrained_layout=True, gridspec_kw={"hspace": 0.1})
axes = axes.flatten()

for bezirk, ax in zip(stadtbezirke, axes):
    wege_bezirk_einzeln = wege_mit_stadtbezirk[
        wege_mit_stadtbezirk["Wohnort Stadtbezirk Text"] == bezirk
    ]
    for mittel in verkehrsmittel:
        wege_mittel = wege_bezirk_einzeln[
            wege_bezirk_einzeln["Hauptverkehrsmittel Gruppierung"] == mittel
        ]["Entfernung in [km] aus Routing GoogleMaps"]
        (wege_mittel).hist(ax=ax, bins=np.linspace(0, 20, 41), label=mittel, histtype="step", color=mittel_to_color[mittel])

    ax.set(ylabel="Anzahl Fahrten", xlabel="Zurückgelegte Kilometer", title=bezirk)

plt.figlegend(verkehrsmittel, bbox_to_anchor=(0.5, 1.05), ncol=6, loc="upper center")

#### Statistik zum Anteil Kfz-Wegen

In [ ]:
wege_kfz_count = wege.loc[(wege["Hauptverkehrsmittel Gruppierung"] == "Kfz")].shape[0]
wege_kfz_kurz_count = wege.loc[(wege["Hauptverkehrsmittel Gruppierung"] == "Kfz") & (wege["Entfernung in [km] aus Routing GoogleMaps"] <= 5)].shape[0]

Anteil Kfz-Wege < 5km

In [ ]:
wege_kfz_kurz_count / wege_kfz_count

Anteil Kfz-Wege an allen Wegen

In [ ]:
wege_kfz_count / wege.shape[0]